In [9]:
import os
import random
import numpy as np
import torch
import pathlib

from sklearn.model_selection import train_test_split
from components.common import load_file_lists

In [10]:
def create_features(features_path, split_path):
    try:
        X_np = np.load(os.path.join(features_path, "X.npy"), allow_pickle=True)
        Y_np = np.load(os.path.join(features_path, "Y.npy"), allow_pickle=True)
    except OSError:
        X = []
        Y = []
        data = load_file_lists([
            os.path.join(split_path, "train.npy"),
            os.path.join(split_path, "valid.npy"),
            os.path.join(split_path, "test.npy")
        ])
        binary = {row[0]: row[1:] for row in np.load(os.path.join(split_path, "binary.npy"), allow_pickle=True)}
        for idx, filename in data:
            filename = os.path.join("../data/mtat/emb", str(pathlib.Path(filename).with_suffix(".npy")))
            file_data = np.load(filename, allow_pickle=True).flatten()
            X.append(file_data)
            Y.append(binary[int(idx)])
        X_np = np.array(X)
        Y_np = np.array(Y)
        np.save(os.path.join(features_path, "X.npy"), X_np)
        np.save(os.path.join(features_path, "y.npy"), Y_np)
    return X_np, Y_np

In [11]:
def set_seed(seed: int = 42) -> None:
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    print(f"Random seed set as {seed}")

In [12]:
set_seed(123456)

Random seed set as 123456


In [13]:
dataset_name = "mtat-10"
split_path = "../split"
features_path = "../data/mtat/features/"

split_path = os.path.join(split_path, dataset_name)
features_path = os.path.join(features_path, dataset_name)
os.makedirs(split_path, exist_ok=True)
os.makedirs(features_path, exist_ok=True)

In [14]:
X_np, Y_np = create_features(features_path, split_path)

In [15]:
X_train, X_test, Y_train, Y_test = train_test_split(X_np, Y_np, random_state=1)

In [16]:
X_train

array([[2.4309678 , 2.4448533 , 3.1502523 , ..., 0.85035396, 2.9974988 ,
        3.2850015 ],
       [2.4309678 , 1.9828519 , 3.3692758 , ..., 1.6137261 , 2.9836583 ,
        3.2943852 ],
       [2.4978356 , 2.1299543 , 3.7867532 , ..., 0.62893057, 3.4161572 ,
        2.7701974 ],
       ...,
       [2.4309678 , 1.7141281 , 4.028397  , ..., 1.3595996 , 3.1440217 ,
        1.9053471 ],
       [2.4309678 , 2.0564501 , 3.5084522 , ..., 1.5454353 , 3.202906  ,
        1.5542096 ],
       [1.3508505 , 0.7575466 , 1.0809858 , ..., 0.34313738, 3.5183105 ,
        1.4912078 ]], dtype=float32)

In [17]:
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(12352, 15360)
(4118, 15360)
(12352, 10)
(4118, 10)


In [18]:
import pickle
from sklearn.exceptions import UndefinedMetricWarning
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier

import warnings
warnings.simplefilter(action='ignore', category=UndefinedMetricWarning)

model = KNeighborsClassifier()
model.fit(X_train, Y_train)
y_pred = model.predict(X_test)
print(classification_report(y_pred, Y_test))

              precision    recall  f1-score   support

           0       0.73      0.82      0.77      1043
           1       0.80      0.75      0.77      1166
           2       0.41      0.64      0.50       583
           3       0.77      0.74      0.75       771
           4       0.51      0.59      0.55       592
           5       0.45      0.58      0.50       492
           6       0.40      0.54      0.46       461
           7       0.79      0.87      0.83       499
           8       0.44      0.46      0.45       562
           9       0.54      0.83      0.66       347

   micro avg       0.60      0.70      0.65      6516
   macro avg       0.58      0.68      0.62      6516
weighted avg       0.62      0.70      0.65      6516
 samples avg       0.65      0.70      0.63      6516



In [19]:
model_filename_path = os.path.join("../models", model.__class__.__name__)
os.makedirs(model_filename_path, exist_ok=True)
model_filename = os.path.join(model_filename_path, f"{dataset_name}.bin")
pickle.dump(model, open(model_filename, 'wb+'))

In [20]:
loaded_model = pickle.load(open("../models/KNeighborsClassifier/mtat-10.bin", 'rb'))
y_pred = loaded_model.predict(X_test)
print(classification_report(y_pred, Y_test))

              precision    recall  f1-score   support

           0       0.73      0.82      0.77      1043
           1       0.80      0.75      0.77      1166
           2       0.41      0.64      0.50       583
           3       0.77      0.74      0.75       771
           4       0.51      0.59      0.55       592
           5       0.45      0.58      0.50       492
           6       0.40      0.54      0.46       461
           7       0.79      0.87      0.83       499
           8       0.44      0.46      0.45       562
           9       0.54      0.83      0.66       347

   micro avg       0.60      0.70      0.65      6516
   macro avg       0.58      0.68      0.62      6516
weighted avg       0.62      0.70      0.65      6516
 samples avg       0.65      0.70      0.63      6516



In [24]:
import pandas as pd
from src.components.preprocessor import OpenL3PreProcessor
from src.components.common import Config


class SklearnPredictor:
    def __init__(self, config: Config, model_filename: str = None, cuda: bool = None):
        if config is None:
            config = Config()

        tags_path = os.path.join(config.dataset_split_path, config.dataset_name, "tags.npy")
        self.tags = np.load(tags_path, allow_pickle=True)

        self.model_filename_path = config.model_filename_path
        self.model_filename = model_filename

        # cuda
        self.is_cuda = torch.cuda.is_available() if cuda is None else cuda

        self.data_path = config.data_path
        self.input_length = config.input_length

        self.preprocessor = OpenL3PreProcessor()

    def predict_tags(self, mp3_file, model=None) -> pd.DataFrame:
        if model is None:
            model = pickle.load(open(os.path.join(self.model_filename_path, self.model_filename), "rb"))

        out = self.predict_file(mp3_file, model).T
        df = pd.DataFrame(out, index=self.tags)
        df = df.reindex(df.mean(axis=1).sort_values(ascending=False).index)
        return df

    def predict_file(self, mp3_file, model=None):
        if model is None:
            model = pickle.load(open(os.path.join(self.model_filename_path, self.model_filename), "rb"))

        return self.predict_data(self.preprocessor.process(mp3_file), model)

    def predict_data(self, x, model=None):
        if model is None:
            model = pickle.load(open(os.path.join(self.model_filename_path, self.model_filename), "rb"))

        return model.predict(x.reshape(1, -1))

In [25]:
file = "../data/mtat/mp3/6/barry_phillips-trad-03-bisonpolska-88-117.mp3"
predictor = SklearnPredictor(Config(dataset_split_path="../split", model_filename_path="../models", dataset_name="mtat-10"),
                             model_filename="KNeighborsClassifier/mtat-10/model.bin")

In [26]:
predictor.predict_file(file)

/home/robak/.virtualenvs/labelthissongforme/lib/python3.10/site-packages/torchopenl3/models.py:200: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  mel_basis = librosa.filters.mel(


array([[0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0]])